<a href="https://colab.research.google.com/github/iamflorenz08/Erovoutika-LMS/blob/main/HybridFiltering_12_16_1512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightfm
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2ZfIgpyBRspxkmpvlT7DIBUaJJL_K6q9YWd5cxCGy3NBowBd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808328 sha256=f85ac680bbefcf47b737307bd4fdd4495af022bbb9238d488ffb3e646f51982b
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=bbe08c9a6a8235962909219f789c6e50a6d1e3a3c67166779b7e7f185dada30b
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
################################################
# Importing necessary library
################################################
import numpy as np
import pandas as pd

# all lightfm imports
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm import cross_validation
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.evaluation import recall_at_k
from lightfm.evaluation import reciprocal_rank

# imports re for text cleaning
import re
from datetime import datetime, timedelta

import warnings
import random
warnings.filterwarnings('ignore')

**Datasets**

In [ ]:
df_topics = pd.read_csv('/content/drive/MyDrive/Datasets/back_up_topics.csv')
df_users = pd.read_csv('/content/drive/MyDrive/Datasets/users.csv')
df_views = pd.read_csv('/content/drive/MyDrive/Datasets/back_up_views.csv')

In [ ]:
df_views = df_views.drop(['Unnamed: 0'],axis=1)
tags_columns = df_topics.filter(like='tags')
df_topics['tags'] = tags_columns.apply(lambda row: ','.join(filter(lambda x: pd.notna(x), row)), axis=1)
df_topics = df_topics[['_id', 'author', 'title', 'tags']]
df_users = df_users.rename(columns={'Tags': 'tags'})

In [ ]:
df_topics['tags'] = df_topics['tags'].str.replace('<', '')
df_topics['tags']  = df_topics['tags'].str.replace('>', ' ')
df_topics['tags']  = df_topics['tags'].str.strip()
df_topics['tags']  = df_topics['tags'].str.replace(' ', ',')

In [ ]:
def biased_random_choice(biased_num):
    random_number = random.random()
    if random_number < biased_num:
        return 1
    else:
        return 0

user_num = []
question_num = []
topics_tags = []
users_tags = []

for tags in df_topics['Tags']:
  topics_tags.append(tags)

for tags in df_users['Tags']:
  users_tags.append(tags.split(','))

for i in range(6):
  for index, topic_tags in enumerate(topics_tags):
    if(biased_random_choice(0.05)):
      user_num.append(i)
      question_num.append(index)
      continue
    else:
      if(biased_random_choice(0.3)):
        for user_tag in users_tags[i]:
          if user_tag in topic_tags:
            user_num.append(i)
            question_num.append(index)
            break

df_views_ = pd.DataFrame()
df_views_['user_num'] = user_num
df_views_['question_num'] = question_num

df_views = df_views_

KeyError: ignored

In [ ]:
def generate_int_id(dataframe, id_col_name):
    """
    Generate unique integer id for users, questions and answers

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A.
    id_col_name : String
        New integer id's column name.

    Returns
    -------
    Dataframe
        Updated dataframe containing new id column
    """
    new_dataframe=dataframe.assign(
        int_id_col_name=np.arange(len(dataframe))
        ).reset_index(drop=True)
    return new_dataframe.rename(columns={'int_id_col_name': id_col_name})



def create_features(dataframe, features_name, id_col_name):
    """
    Generate features that will be ready for feeding into lightfm

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe which contains features
    features_name : List
        List of feature columns name avaiable in dataframe
    id_col_name: String
        Column name which contains id of the question or
        answer that the features will map to.
        There are two possible values for this variable.
        1. questions_id_num
        2. professionals_id_num

    Returns
    -------
    Pandas Series
        A pandas series containing process features
        that are ready for feed into lightfm.
        The format of each value
        will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
        Ex. -> (1, ['military', 'army', '5'])
    """

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features



def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A.
    features_name : List
        List of feature columns name avaiable in dataframe.

    Returns
    -------
    List of all features for mapping
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features


def calculate_auc_score(lightfm_model, interactions_matrix,
                        question_features, professional_features):
    """
    Measure the ROC AUC metric for a model.
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model
        A fitted lightfm model
    interactions_matrix :
        A lightfm interactions matrix
    question_features, professional_features:
        Lightfm features

    Returns
    -------
    String containing AUC score
    """
    score = auc_score(
        lightfm_model, interactions_matrix,
        item_features=question_features,
        user_features=professional_features,
        num_threads=4).mean()
    return score

In [ ]:
# generating unique integer id for users and q&a
df_users = generate_int_id(df_users, 'users_id_num')
df_topics = generate_int_id(df_topics, 'topics_id_num')
df_views = generate_int_id(df_views, 'views_id_num')

In [ ]:
df_users = df_users.rename(columns={'tags': 'user_tags'})
df_topics = df_topics.rename(columns={'tags': 'topic_tags'})
df_views = df_views.rename(columns={'user_num': 'user_id'})
df_views = df_views.rename(columns={'question_num': 'topic_id'})

In [ ]:
df_merge = df_views.merge(
    df_users, how='left',
    left_on='user_id', right_on='users_id_num')
df_merge = df_merge.merge(
    df_topics, how='left',
    left_on='topic_id', right_on='topics_id_num')

In [ ]:
viewed_topics_tags = df_merge[['UserId', 'topic_tags']]

viewed_topics_tags = viewed_topics_tags.groupby(
    ['UserId'])['topic_tags'].apply(
        ','.join).reset_index()

viewed_topics_tags['topic_tags'] = (
    viewed_topics_tags['topic_tags'].str.split(',').apply(set).str.join(','))

df_users = df_users.merge(viewed_topics_tags, how='left', on='UserId')

df_users['user_all_tags'] = (
    df_users[['user_tags', 'topic_tags']].apply(
        lambda x: ','.join(x.dropna()),
        axis=1))

In [ ]:
df_topics['topic_tags'] = df_topics['topic_tags'].str.split(',').apply(set).str.join(',')
df_users['user_all_tags'] = df_users['user_all_tags'].fillna('No Tag')
df_users['user_all_tags'] = df_users['user_all_tags'].replace('', 'No Tag')
df_users['user_all_tags'] = df_users['user_all_tags'].str.split(',').apply(set).str.join(',')

In [ ]:
# generating features list for mapping
topic_feature_list = generate_feature_list(
    df_topics,
    ['topic_tags'])

user_feature_list = generate_feature_list(
    df_users,
    ['user_all_tags'])

In [ ]:
# creating features for feeding into lightfm
df_topics['topic_features'] = create_features(
    df_topics, ['topic_tags'],
    'topics_id_num')

df_users['user_features'] = create_features(
    df_users,
    ['user_all_tags'],
    'users_id_num')

In [ ]:
########################
# Dataset building for lightfm
########################

# define our dataset variable
# then we feed unique professionals and questions ids
# and item and professional feature list
# this will create lightfm internel mapping
dataset = Dataset()
dataset.fit(
    set(df_users['users_id_num']),
    set(df_topics['topics_id_num']),
    item_features=topic_feature_list,
    user_features=user_feature_list)


# now we are building interactions matrix between professionals and quesitons
# we are passing professional and questions id as a tuple
# e.g -> pd.Series((pro_id, question_id), (pro_id, questin_id))
# then we use lightfm build in method for building interactions matrix
df_merge['user_topic_tuple'] = list(zip(
    df_merge.users_id_num, df_merge.topics_id_num))

interactions, weights = dataset.build_interactions(
    df_merge['user_topic_tuple'])

train_interaction, test_interaction = cross_validation.random_train_test_split(interactions, 0.3, np.random.RandomState(2019))
train_weights, test_weights = cross_validation.random_train_test_split(interactions, 0.3, np.random.RandomState(2019))

# now we are building our questions and professionals features
# in a way that lightfm understand.
# we are using lightfm build in method for building
# questions and professionals features
topic_features = dataset.build_item_features(
    df_topics['topic_features'])

user_features = dataset.build_user_features(
    df_users['user_features'])


In [ ]:
################################
# Model building part
################################

# define lightfm model by specifying hyper-parametre
# then fit the model with ineteractions matrix, item and user features
model = LightFM(
    no_components=350,
    learning_rate=0.1,
    loss='warp',
    k=15,
    random_state=2019)

model.fit(
    interactions,
    item_features=topic_features,
    user_features=user_features,
    sample_weight=interactions,
    epochs=45, num_threads=4, verbose=True)

Epoch: 100%|██████████| 45/45 [00:01<00:00, 22.77it/s]


In [ ]:
print('AUC Score', auc_score(model, interactions, user_features=user_features, item_features=topic_features).mean())
print('Precision at 15 Score',precision_at_k(model, interactions,user_features=user_features, item_features=topic_features, k=15).mean())
print('Recall at 15 Score',recall_at_k(model, interactions,user_features=user_features, item_features=topic_features, k=15).mean())
print('Reciprocal rank Score',reciprocal_rank(model, interactions,user_features=user_features, item_features=topic_features).mean())


AUC Score 0.9951532
Precision at 15 Score 0.8777778
Recall at 15 Score 0.8007725682129854
Reciprocal rank Score 0.8888889


In [ ]:
def predict(user_id, items_per_page=10):
  matching_indices = df_users.loc[df_users['UserId'].str.strip() == user_id].index
  user_index = int(matching_indices[0])

  previous_topic_id_num = df_merge.loc[df_merge['users_id_num'] == user_index][:10]['topics_id_num']
  df_previous_topics = df_topics.loc[df_topics['topics_id_num'].isin(previous_topic_id_num)]

  discard_topic_id = df_previous_topics['topics_id_num'].values.tolist()
  df_use_for_prediction = df_topics.loc[~df_topics['topics_id_num'].isin(discard_topic_id)]
  questions_id_for_predict = df_use_for_prediction['topics_id_num'].values.tolist()

  scores = model.predict(
              user_index,
              questions_id_for_predict,
              item_features=topic_features,
              user_features=user_features)

  df_use_for_prediction['scores'] = scores
  df_use_for_prediction = df_use_for_prediction.sort_values(by='scores', ascending=False)[:items_per_page]
  return df_use_for_prediction['_id']

In [ ]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/recommend/<string:user_id>")
def home(user_id):
    per_page = int(request.args.get('per_page', 10))
    return jsonify(predict(user_id, per_page).to_numpy().tolist())


app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f93f-34-82-131-196.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 03:03:05] "GET /recommend/656d5d64a6dc8e108c76a5c9?per_page=30 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 03:03:05] "GET /recommend/656d5d64a6dc8e108c76a5c9?per_page=30 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 03:04:35] "GET /recommend/656d5d64a6dc8e108c76a5c9?per_page=30 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 03:07:16] "GET /recommend/657be5917bf6fa0cb36a2adb?per_page=30 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 03:09:23] "GET /recommend/657be5917bf6fa0cb36a2adb?per_page=30 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 03:29:42] "GET /recommend/656d5d64a6dc8e108c76a5c9?per_page=30 HTTP/1.1" 200 -
